In [1]:
# nltk.download('stopwords')
# nltk.download('wordnet')
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import string
import nltk
import it_core_news_sm
import re
from itertools import combinations
import itertools
import os
import networkx as nx

from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
from wordcloud import WordCloud


In [2]:
database_words = pd.read_csv("~/Scrivania/common_start/database/id_lemmas.csv",index_col = 0 , sep=',', engine='python')
database_words['text_nlp'] = database_words.apply(lambda row: word_tokenize(row['text_nlp']), axis = 1)
database_words

,text_nlp
0,"[diro, gente, guerra, portato, casa, rimpatriare]"
1,"[marcello, perfavore]"
2,"[patrio, patriota, difeso, proprio, radice, so..."
3,"[musulmano, comandare, casa]"
4,"[odiare, dipendere, odiare, comandare, librare]"
...,...
78170,"[vedere, ruspa, inviato, salvini, ripulire, am..."
78171,"[fino, sentire, sempre, richiamare, africo, pa..."
78172,"[sparare, ladro, occhio, penna, barbetta]"
78173,"[consigliatissima, africo]"


In [3]:
database_comments = pd.read_csv("~/Scrivania/common_start/database/com_liwc.csv", sep='\t', engine='python')
database_comments.head(2)

,Origin_file_order,Site,p_id,dateCreated,p_politician,p_gender,p_GRUPPO_PE,p_LISTA,p_PARTITO,p_governo,...,p_Comma,p_Colon,p_SemiC,p_Qmark,p_Exclam,p_Dash,p_Quote,p_Apostro,p_Parenth,p_OtherP
0,30126,FB,96844400700_10157493758850701,2019-04-23T10:33:37Z,MARCELLO GEMMATO,M,PPE,FDI,FDI,opposizione,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,30226,FB,96844400700_10157493758850701,2019-04-23T10:33:37Z,MARCELLO GEMMATO,M,PPE,FDI,FDI,opposizione,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# THE FOLLOWING CELLS ARE THE ONES TO BE MODIFIED IN ORDER TO OBTAIN ANY OTHER SUBSET

## Here keep only problematic and hate comments

In [4]:
subset_comments = database_comments[database_comments.loc[:,'c_rating3'] == 'probl-hate'].loc[:,'c_rating3']
#free memory
subset_comments

0        probl-hate
1        probl-hate
2        probl-hate
3        probl-hate
4        probl-hate
            ...    
78043    probl-hate
78044    probl-hate
78045    probl-hate
78046    probl-hate
78047    probl-hate
Name: c_rating3, Length: 6636, dtype: object

computes the maching indexes and retrieve the subset from the lemmas dataset

In [5]:
matching_indexes = database_words.index.intersection(subset_comments.index)
subset_words = database_words.loc[matching_indexes,:]
subset_words

,text_nlp
0,"[diro, gente, guerra, portato, casa, rimpatriare]"
1,"[marcello, perfavore]"
2,"[patrio, patriota, difeso, proprio, radice, so..."
3,"[musulmano, comandare, casa]"
4,"[odiare, dipendere, odiare, comandare, librare]"
...,...
78043,"[folla, quell, handicappare, linguacciuto, gio..."
78044,"[cinghiare, morono, terra]"
78045,"[farli, prendere, cinghiare, terra, fermo]"
78046,"[immigrato, rompere, coglione]"


In [6]:
##check for null values
subset_words.isnull().values.any()

False

In [7]:
##create a copy of the dataframe and add the index
df_words = subset_words.copy()
df_words['id'] = df_words.index
df_words

,text_nlp,id
0,"[diro, gente, guerra, portato, casa, rimpatriare]",0
1,"[marcello, perfavore]",1
2,"[patrio, patriota, difeso, proprio, radice, so...",2
3,"[musulmano, comandare, casa]",3
4,"[odiare, dipendere, odiare, comandare, librare]",4
...,...,...
78043,"[folla, quell, handicappare, linguacciuto, gio...",78043
78044,"[cinghiare, morono, terra]",78044
78045,"[farli, prendere, cinghiare, terra, fermo]",78045
78046,"[immigrato, rompere, coglione]",78046


Create a dataframe of unique words

In [8]:
unique_words = {}
words_per_comment = {}

for index, row in df_words.iterrows():
    if(len(row['text_nlp'])>0):
        words_per_comment[row["id"]]= [clean_txt for clean_txt in row['text_nlp']] ###iterating the column text_clean. There are 78174 rows
        for single_word in row['text_nlp']:
            unique_words.setdefault(single_word, 0) ## setdefault() method returns the value of a key (if the key is in dictionary)
            unique_words[single_word] += 1

#len of unique words
print("Number of unique words: ", len(unique_words))
unique_words

Number of unique words:  11008


{'diro': 125,
 'gente': 172,
 'guerra': 94,
 'portato': 18,
 'casa': 330,
 'rimpatriare': 20,
 'marcello': 3,
 'perfavore': 1,
 'patrio': 43,
 'patriota': 6,
 'difeso': 29,
 'proprio': 33,
 'radice': 13,
 'soccombere': 1,
 'roma': 36,
 'islamico': 90,
 'accoltellare': 3,
 'uomo': 54,
 'crocifisso': 9,
 'collare': 11,
 'accadere': 14,
 'stazione': 7,
 'termine': 12,
 'aggressore': 1,
 'fermare': 25,
 'poliziotto': 13,
 'notare': 12,
 'vittima': 16,
 'tentare': 7,
 'sgozzarlo': 1,
 'aumentare': 18,
 'controllo': 6,
 'attenzione': 17,
 'prevenire': 2,
 'violenza': 27,
 'cittadino': 47,
 'innocente': 11,
 'partire': 166,
 'imperativo': 1,
 'corrado': 4,
 'armeri': 2,
 'fdi': 7,
 'musulmano': 39,
 'comandare': 32,
 'odiare': 65,
 'dipendere': 12,
 'librare': 12,
 'dire': 207,
 'dovere': 142,
 'diventare': 43,
 'razzista': 72,
 'peggio': 38,
 'meditato': 8,
 'sinistronzi': 11,
 'tolleranza': 9,
 'rispettare': 76,
 'cultura': 54,
 'leggere': 141,
 'barcone': 24,
 'via': 142,
 'sottoscrivere':

In [9]:
folder_path = 'subsets/'
os.makedirs(folder_path,exist_ok=True)

## Create the unique words dataframe

Moreover as attribute it has how many times it appears in a given network, in this case hate comments

In [10]:
lemmas_df        = pd.DataFrame.from_dict(unique_words, orient='index').reset_index()
lemmas_df.columns = ['id', 'counts'] 
lemmas_df

,id,counts
0,diro,125
1,gente,172
2,guerra,94
3,portato,18
4,casa,330
...,...,...
11003,handicappare,1
11004,linguacciuto,1
11005,morono,1
11006,elettrizzare,1


In [11]:
#export
lemmas_df.to_csv('subsets/NODES_hate_problematic_subset.csv', index = False)

In [12]:
##create a matrix of words
clean_df = pd.DataFrame(0, index=unique_words, columns=unique_words)

for key in words_per_comment:
    for pair in itertools.product(words_per_comment[key],words_per_comment[key]):
        if pair[0]!=pair[1] and not(clean_df.at[pair[0],pair[1]]):
            clean_df.at[pair[0],pair[1]] += 1
            clean_df.at[pair[1],pair[0]] += 1

display(clean_df) ##this is the full matrix of words

,diro,gente,guerra,portato,casa,rimpatriare,marcello,perfavore,patrio,patriota,...,malauguratamente,seggiolina,incendio,causale,rimpatriamoli,handicappare,linguacciuto,morono,elettrizzare,dalleparoleaifatti
diro,0,1,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
gente,1,0,1,1,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
guerra,1,1,0,1,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,1
portato,1,1,1,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
casa,1,1,1,1,0,1,0,0,1,0,...,0,0,1,1,1,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
handicappare,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
linguacciuto,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
morono,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
elettrizzare,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [13]:
##use this as a basis to get edges and their weights

words_projection = {} ##create dictionary 
##itertools.product() which computes the cartesian product of input iterables.

for key in words_per_comment:
    for pair in itertools.product(words_per_comment[key],words_per_comment[key]):
        if pair[0]!=pair[1] and not(pair[::-1] in words_projection):
            words_projection.setdefault(pair,0)
            words_projection[pair] += 1

words_projection

{('diro', 'gente'): 9,
 ('diro', 'guerra'): 6,
 ('diro', 'portato'): 2,
 ('diro', 'casa'): 13,
 ('diro', 'rimpatriare'): 1,
 ('gente', 'guerra'): 2,
 ('gente', 'portato'): 1,
 ('gente', 'casa'): 20,
 ('gente', 'rimpatriare'): 1,
 ('guerra', 'portato'): 6,
 ('guerra', 'casa'): 10,
 ('guerra', 'rimpatriare'): 2,
 ('portato', 'casa'): 5,
 ('portato', 'rimpatriare'): 1,
 ('casa', 'rimpatriare'): 1,
 ('marcello', 'perfavore'): 1,
 ('patrio', 'patriota'): 2,
 ('patrio', 'difeso'): 1,
 ('patrio', 'proprio'): 14,
 ('patrio', 'radice'): 1,
 ('patrio', 'soccombere'): 1,
 ('patrio', 'roma'): 4,
 ('patrio', 'islamico'): 8,
 ('patrio', 'accoltellare'): 1,
 ('patrio', 'uomo'): 4,
 ('patrio', 'crocifisso'): 1,
 ('patrio', 'collare'): 1,
 ('patrio', 'accadere'): 5,
 ('patrio', 'stazione'): 1,
 ('patrio', 'termine'): 1,
 ('patrio', 'aggressore'): 1,
 ('patrio', 'fermare'): 1,
 ('patrio', 'poliziotto'): 1,
 ('patrio', 'notare'): 1,
 ('patrio', 'vittima'): 3,
 ('patrio', 'tentare'): 1,
 ('patrio', 'sgozz

In [14]:
##obtain weighted edges list
##edge lists =36million
#WEIGHTED
#to get weighted graph we need a list of 3-element tuplels (u,v,w) where u and v are nodes 
## and w is a number representing weight

words_weighted = []
for edge in words_projection:
    words_weighted.append((edge[0],edge[1],words_projection[edge]))

G = nx.Graph()
G.add_weighted_edges_from(words_weighted)
nx.write_weighted_edgelist(G, "subsets/EDGES_hate_problematic_subset.csv",delimiter=",") ##save the edges list as csv
